In [2]:
# coding: utf-8
from glob import glob
import os.path as op
import pandas as pd
import numpy as np
from bids.layout import BIDSLayout

out_dir = '/home/data/nbc/Sutherland_HIVCB/EAT-behavioral/'
dset_dir = '/home/data/nbc/Sutherland_HIVCB/dset/'
layout = BIDSLayout(dset_dir)
subjects = layout.get_subjects()
#subjects = subjects[:5]
#subjects.remove('283')
#subjects.remove('397')
#subjects.remove('406')

In [3]:
out_df = pd.DataFrame(
    columns=['n_correct_incongruent', 'n_incorrect_incongruent', 'n_incongruent',
             'n_correct_repeat', 'n_incorrect_repeat', 'n_repeat',
             'n_correct_nogo', 'n_incorrect_go', 'n_incorrect_nogo', 'n_nogo',
             'n_correct_go', 'n_incorrect_go', 'n_go', 'n_nogo_aware', 
             'n_nogo_unaware', 'n_nogo_noresponse',
             'mean_pre_correct_rt', 'mean_post_correct_rt',
             'mean_aware_stroop_rt', 'mean_aware_repeat_rt',
             'mean_unaware_stroop_rt', 'mean_unaware_repeat_rt',
             'mean_pre_aware_rt', 'mean_post_aware_rt',
             'mean_pre_unaware_rt', 'mean_post_unaware_rt',
             'mean_nogo_aware_rt', 'mean_nogo_unaware_rt', 
             'mean_incorrect_incongruent_rt', 'std_incorrect_incongruent_rt',
             'mean_incongruent_rt', 'std_incongruent_rt',
             'mean_incorrect_repeat_rt', 'std_incorrect_repeat_rt',
             'mean_repeat_rt', 'std_repeat_rt',
             'mean_incorrect_nogo_rt', 'std_incorrect_nogo_rt', 
             'mean_correct_nogo_rt', 'std_correct_nogo_rt',
             'mean_correct_go_rt', 'std_correct_go_rt',
             'n_aware_repeat', 'n_unaware_repeat', 'n_aware_stroop', 'n_unaware_stroop',
             'mean_incorrect_go_rt', 'std_incorrect_go_rt', 'mean_onepost_aware_rt', 'mean_onepost_unaware_rt'],
    index=subjects)

for subject in subjects:
    files = sorted(glob(op.join(dset_dir, 'sub-'+subject+'/func/sub-*_task-errorawareness*events.tsv')))
    if not files:
        print('{0} failed'.format(subject))
        continue
    else:
        print('{0} running'.format(subject))

    dfs = [pd.read_csv(f, sep='\t') for f in files]
    df = pd.concat(dfs)
    df['go'] = df['trial_type_2'].str.startswith('go').astype(int)
    #df['nogo_error'] = df['trial_type_2'].str.startswith('nogoIncorrect').astype(int)

    acc_df1 = df.groupby(['trial_type', 'trial_accuracy']).count()
    acc_df2 = df.groupby(['trial_type']).count()
    acc_df3 = df.groupby(['trial_type_2', 'trial_accuracy']).count()
    acc_df4 = df.groupby(['trial_type_2']).count()
    acc_df5 = df.groupby(['trial_type_2']).count()
    acc_df6 = df.groupby(['trial_type_3']).count()

    rt_df1 = df.groupby(['trial_type', 'trial_accuracy']).mean()
    rt_df2 = df.groupby(['trial_type']).mean()
    rt_df3 = df.groupby(['trial_type_2', 'trial_accuracy']).mean()
    rt_df4 = df.groupby(['trial_type_2']).mean()
    rt_df5 = df.groupby(['trial_type', 'trial_accuracy']).std()
    rt_df6 = df.groupby(['trial_type']).std()
    rt_df7 = df.groupby(['trial_type_2', 'trial_accuracy']).std()
    rt_df8 = df.groupby(['trial_type_2']).std()
    rt_df9 = df.groupby(['trial_type_2']).mean()
    rt_df12 = df.groupby(['trial_type_3']).mean()
    
    
    
    df['all_nogoIncorrect'] = df['trial_type_2'].str.startswith('nogoIncorrect')
    rt_df10 = df.groupby(['all_nogoIncorrect']).mean()
    rt_df11 = df.groupby(['all_nogoIncorrect']).std()
    
    try:
        out_df.loc[subject, 'n_correct_incongruent'] = acc_df1.loc[('incongruent', 1), 'onset']
    except:
        out_df.loc[subject, 'n_correct_incongruent'] = 0

    try:
        out_df.loc[subject, 'n_incorrect_incongruent'] = acc_df1.loc[('incongruent', 0), 'onset']
        out_df.loc[subject, 'mean_incorrect_incongruent_rt'] = rt_df1.loc[('incongruent', 0), 'response_time']
        out_df.loc[subject, 'std_incorrect_incongruent_rt'] = rt_df5.loc[('incongruent', 0), 'response_time']
    except:
        out_df.loc[subject, 'n_correct_incongruent'] = 0
        out_df.loc[subject, 'mean_incorrect_incongruent_rt'] = np.NaN
        out_df.loc[subject, 'std_incorrect_incongruent_rt'] = np.NaN

    try:
        out_df.loc[subject, 'n_correct_repeat'] = acc_df1.loc[('repeat', 1), 'onset']
    except:
        out_df.loc[subject, 'n_correct_repeat'] = 0

    try:
        out_df.loc[subject, 'n_incorrect_repeat'] = acc_df1.loc[('repeat', 0), 'onset']
        out_df.loc[subject, 'mean_incorrect_repeat_rt'] = rt_df1.loc[('repeat', 0), 'response_time']
        out_df.loc[subject, 'std_incorrect_repeat_rt'] = rt_df5.loc[('repeat', 0), 'response_time']
    except:
        out_df.loc[subject, 'n_incorrect_repeat'] = 0
        out_df.loc[subject, 'mean_incorrect_repeat_rt'] = np.NaN
        out_df.loc[subject, 'std_incorrect_repeat_rt'] = np.NaN

    out_df.loc[subject, 'n_correct_nogo'] = df.loc[df['trial_type_2'] == 'nogoCorrect'].shape[0]
    out_df.loc[subject, 'n_correct_go'] = df.loc[df['trial_type_2'] == 'goCorrect'].shape[0]
    out_df.loc[subject, 'n_incorrect_nogo'] = df.loc[df['trial_type_2'].str.startswith('nogoIncorrect')].shape[0]
    out_df.loc[subject, 'n_correct_nogo'] = df.loc[df['trial_type_2'] == 'nogoCorrect'].shape[0]
    out_df.loc[subject, 'n_incorrect_go'] = df.loc[df['trial_type_2'] == 'goIncorrect'].shape[0]
    out_df.loc[subject, 'n_nogo_aware'] = df.loc[df['trial_type_2'] == 'nogoIncorrectAware'].shape[0]
    out_df.loc[subject, 'n_nogo_unaware'] = df.loc[df['trial_type_2'] == 'nogoIncorrectUnaware'].shape[0]
    out_df.loc[subject, 'n_nogo_noresponse'] = df.loc[df['trial_type_2'].str.startswith('nogoIncorrectNoResponse')].shape[0]
    #out_df.loc[subject, 'n_nogo'] = df.loc[df['trial_type_2'].str.startswith('nogo')].shape[0]
    #out_df.loc[subject, 'n_go'] = df.loc[df['trial_type_2'].str.startswith('go')].shape[0]
    
    try:
        out_df.loc[subject, 'mean_incorrect_nogo_rt'] = rt_df10.loc[1, 'response_time']
        out_df.loc[subject, 'std_incorrect_nogo_rt'] = rt_df11.loc[1, 'response_time']
    except:
        out_df.loc[subject, 'mean_incorrect_nogo_rt'] = np.NaN
        out_df.loc[subject, 'std_incorrect_nogo_rt'] = np.NaN
        
    try:
        out_df.loc[subject, 'mean_correct_nogo_rt'] = rt_df9.loc['nogoCorrect', 'response_time']
        out_df.loc[subject, 'std_correct_nogo_rt'] = rt_df8.loc['nogoCorrect', 'response_time']
    except:
        out_df.loc[subject, 'mean_correct_nogo_rt'] = np.NaN
        out_df.loc[subject, 'std_correct_nogo_rt'] = np.NaN

    try:
        out_df.loc[subject, 'mean_correct_go_rt'] = rt_df9.loc['goCorrect', 'response_time']
        out_df.loc[subject, 'std_correct_go_rt'] = rt_df8.loc['goCorrect', 'response_time']
    except:
        out_df.loc[subject, 'mean_correct_go_rt'] = np.NaN
        out_df.loc[subject, 'std_correct_go_rt'] = np.NaN

    try:
        out_df.loc[subject, 'mean_incorrect_go_rt'] = rt_df9.loc['goIncorrect', 'response_time']
        out_df.loc[subject, 'std_incorrect_go_rt'] = rt_df8.loc['goIncorrect', 'response_time']
    except:
        out_df.loc[subject, 'mean_incorrect_go_rt'] = np.NaN
        out_df.loc[subject, 'std_incorrect_go_rt'] = np.NaN
        
    try:
        out_df.loc[subject, 'mean_nogo_aware_rt'] = rt_df9.loc['nogoIncorrectAware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_aware'] = 0
        out_df.loc[subject, 'mean_nogo_aware_rt'] = np.NaN
        
    try:
        out_df.loc[subject, 'mean_nogo_unaware_rt'] = rt_df9.loc['nogoIncorrectUnaware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_unaware'] = 0
        out_df.loc[subject, 'mean_nogo_unaware_rt'] = np.NaN
    
    
    try:
        out_df.loc[subject, 'mean_aware_stroop_rt'] = rt_df12.loc['stroopIncorrectAware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_aware'] = 0
        out_df.loc[subject, 'mean_aware_stroop_rt'] = np.NaN 
        
    try:
        out_df.loc[subject, 'mean_aware_repeat_rt'] = rt_df12.loc['repeatIncorrectAware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_aware'] = 0
        out_df.loc[subject, 'mean_aware_repeat_rt'] = np.NaN
        
    try:
        out_df.loc[subject, 'mean_unaware_stroop_rt'] = rt_df12.loc['stroopIncorrectUnaware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_unaware'] = 0
        out_df.loc[subject, 'mean_unaware_stroop_rt'] = np.NaN 
        
    try:
        out_df.loc[subject, 'mean_unaware_repeat_rt'] = rt_df12.loc['repeatIncorrectUnaware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_unaware'] = 0
        out_df.loc[subject, 'mean_unaware_repeat_rt'] = np.NaN
    
    try:
        out_df.loc[subject, 'mean_pre_aware_rt'] = rt_df12.loc['preIncorrectAware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_aware'] = 0
        out_df.loc[subject, 'mean_pre_aware_rt'] = np.NaN 
           
    try:
        out_df.loc[subject, 'mean_post_aware_rt'] = rt_df12.loc['postIncorrectAware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_aware'] = 0
        out_df.loc[subject, 'mean_post_aware_rt'] = np.NaN
             
    try:
        out_df.loc[subject, 'mean_onepost_aware_rt'] = rt_df12.loc['onepostIncorrectAware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_aware'] = 0
        out_df.loc[subject, 'mean_onepost_aware_rt'] = np.NaN       
        
    try:
        out_df.loc[subject, 'mean_pre_unaware_rt'] = rt_df12.loc['preIncorrectUnaware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_unaware'] = 0
        out_df.loc[subject, 'mean_pre_unaware_rt'] = np.NaN 
           
    try:
        out_df.loc[subject, 'mean_post_unaware_rt'] = rt_df12.loc['postIncorrectUnaware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_unaware'] = 0
        out_df.loc[subject, 'mean_post_unaware_rt'] = np.NaN 
        
    try:
        out_df.loc[subject, 'mean_onepost_unaware_rt'] = rt_df12.loc['onepostIncorrectUnaware', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_unaware'] = 0
        out_df.loc[subject, 'mean_onepost_unaware_rt'] = np.NaN        
        
        
    try:
        out_df.loc[subject, 'mean_pre_correct_rt'] = rt_df12.loc['pre-nogoCorrect', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_correct'] = 0
        out_df.loc[subject, 'mean_pre_correct_rt'] = np.NaN 
           
    try:
        out_df.loc[subject, 'mean_post_correct_rt'] = rt_df12.loc['post-nogoCorrect', 'response_time']
    except:
        out_df.loc[subject, 'n_nogo_correct'] = 0
        out_df.loc[subject, 'mean_post_correct_rt'] = np.NaN       
        
        

    out_df.loc[subject, 'n_incongruent'] = acc_df2.loc['incongruent', 'onset']
    out_df.loc[subject, 'mean_incongruent_rt'] = rt_df2.loc['incongruent', 'response_time']
    out_df.loc[subject, 'std_incongruent_rt'] = rt_df6.loc['incongruent', 'response_time']

    out_df.loc[subject, 'n_repeat'] = acc_df2.loc['repeat', 'onset']
    out_df.loc[subject, 'mean_repeat_rt'] = rt_df2.loc['repeat', 'response_time']
    out_df.loc[subject, 'std_repeat_rt'] = rt_df6.loc['repeat', 'response_time']
    
    try:
        out_df.loc[subject, 'n_aware_stroop'] = acc_df6.loc['stroopIncorrectAware', 'onset']
    except:
        out_df.loc[subject, 'n_aware_stroop'] = 0
        
    try:
        out_df.loc[subject, 'n_aware_repeat'] = acc_df6.loc['repeatIncorrectAware', 'onset']
    except:
        out_df.loc[subject, 'n_aware_repeat'] = 0  
        
    try:
        out_df.loc[subject, 'n_unaware_repeat'] = acc_df6.loc['repeatIncorrectUnaware', 'onset']
    except:
        out_df.loc[subject, 'n_unaware_repeat'] = 0    

    try:
        out_df.loc[subject, 'n_unaware_stroop'] = acc_df6.loc['stroopIncorrectUnaware', 'onset']
    except:
        out_df.loc[subject, 'n_unaware_stroop'] = 0

    #out_df.loc[subject, 'mean_nogo_rt'] = rt_df4.loc[0, 'response_time']
    #out_df.loc[subject, 'std_nogo_rt'] = rt_df8.loc[0, 'response_time']

    #out_df.loc[subject, 'mean_go_rt'] = rt_df4.loc[1, 'response_time']
    #out_df.loc[subject, 'std_go_rt'] = rt_df8.loc[1, 'response_time']

out_df.to_csv(op.join(out_dir, 'eat_performance.csv'), index_label='subject_id')


193 running
194 running
198 running
203 running
205 running
206 running
211 running
212 running
216 running
217 running
218 running
225 running
229 running
237 running
238 running
240 running
241 running
242 running
244 running
247 running
248 running
250 running
253 running
254 running
257 running
258 running
259 running
263 running
266 running
268 running
273 running
279 running
280 running
282 running
283 running
285 running
287 running
289 running
290 running
291 running
293 running
294 running
295 running
296 failed
297 running
299 running
300 running
301 running
302 running
306 running
307 running
309 running
310 running
311 running
313 running
314 running
315 running
316 running
317 running
318 running
319 running
320 running
321 running
323 running
324 running
326 running
328 running
331 running
332 running
333 running
334 running
336 running
337 running
339 running
340 running
341 running
342 running
343 running
344 running
345 running
346 running
348 running
350 running
351 r